In [1]:
import os
import time
import datetime
import numpy as np
import warnings
import random
from PIL import Image
from tqdm import tqdm

from labels import labels

In [2]:
# 데이터 경로
#  os.getcwd() :'/home/ag/Capstone2/DeepLab_V3_ver1'
origin_data_path = os.getcwd() + '/data/Original_data/'
labeled_data_path = os.getcwd() + '/data/Labeled_data/'

origin_data_list = os.listdir(origin_data_path) # x
labeled_data_list = os.listdir(labeled_data_path) # y

# 파일명 랜덤
random.shuffle(origin_data_list)
random.shuffle(labeled_data_list)

# train, test / x, y 
train_x_file = origin_data_list[:int(len(origin_data_list)*0.8)]
train_y_file = [file_name[:-4] + '_L.png' for file_name in train_x_file]

test_x_file = [file_name for file_name in origin_data_list if file_name not in train_x_file]
test_y_file = [file_name[:-4] + '_L.png' for file_name in test_x_file]

In [3]:
print(len(train_x_file), len(test_x_file), len(train_y_file), len(test_y_file))

560 141 560 141


In [4]:
print(train_x_file[0], test_x_file[0], train_y_file[0], test_y_file[0])

Seq05VD_f02220.png 0016E5_08107.png Seq05VD_f02220_L.png 0016E5_08107_L.png


### X

In [5]:
train_x = [np.array(Image.open(origin_data_path + train)) for train in train_x_file]
test_x = [np.array(Image.open(origin_data_path + test)) for test in test_x_file]

### Y

In [6]:
# color to label catId
color2label = { label.color   : label.id for label in labels}

In [7]:
train_y = []
for file_name in tqdm(train_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    train_y.append(ret)

100%|██████████| 560/560 [21:43<00:00,  2.33s/it]


In [8]:
test_y = []
for file_name in tqdm(test_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    test_y.append(ret)

100%|██████████| 141/141 [05:28<00:00,  2.33s/it]


In [9]:
np.array(train_y).shape, np.array(test_y).shape

((560, 720, 960), (141, 720, 960))

In [10]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

### data save

In [11]:
np.savez('data.npz', train_x=train_x, train_y=train_y, test_x=test_x, test_y=test_y)

# Model

### package load

In [1]:
import os
import sys
import time
import datetime
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
from model import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead, UNet
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### CPU or GPU
#### 아래 코드에서 True이면 GPU 사용, False 이면 CPU 사용됨.

In [3]:
USE_CUDA = torch.cuda.is_available() and False
device = torch.device('cuda' if USE_CUDA else 'cpu')

### data load

In [4]:
npzfile = np.load('data.npz')

train_x = npzfile['train_x']
train_y = npzfile['train_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

### DeepLab v3 ResNet50 

In [5]:
# model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False ).to(device)
model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False).to(device)
model.classifier = DeepLabHead(2048, 12).to(device) # 12 = class num

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) # 파라미터 학습속도 조절

In [7]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [8]:
# Create list of class names
classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

In [9]:
validClasses = list(np.unique([label.id for label in labels if label.id >= 0] + [11]))

### train

In [10]:
model.train()
print()

In [11]:
X = torch.tensor(train_x, dtype=torch.float32)
Y = torch.tensor(train_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

train_data = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True)

In [12]:
X.shape, Y.shape

(torch.Size([560, 720, 960, 3]), torch.Size([560, 720, 960]))

In [13]:
class_weight = torch.tensor([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0], dtype=torch.float32) # 아직 반영안함
criterion = nn.CrossEntropyLoss(weight = None, ignore_index=12) # weight 파라미터에 class_weight 추가

In [14]:
num_epoch = 1
res = X.shape[1] * X.shape[2]

In [15]:
for epoch in range(num_epoch):
    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))
    
    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum())
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
        progress.display(epoch)
        
        
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    # save checkpoint
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d_%H:%M')
    
    # save path
    if not os.path.isdir(os.getcwd() + '/result'):
        os.makedirs(os.getcwd() + '/result')
    
    save_path = os.getcwd() + '/result/'
    
    # Save best model to file
    torch.save({
        'epoch' : epoch,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            }, save_path + now_time + '_best_weights.pth.tar')

        
    print('epoch ', epoch)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

  2%|▏         | 1/56 [01:30<1:22:53, 90.43s/it]

Train, epoch: [0][ 0/56]	Loss 2.5211e+00 (2.5211e+00)	Accuracy 0.103 (0.103)


  4%|▎         | 2/56 [03:01<1:21:26, 90.48s/it]

Train, epoch: [0][ 0/56]	Loss 2.4294e+00 (2.4753e+00)	Accuracy 0.157 (0.130)


  5%|▌         | 3/56 [04:31<1:19:56, 90.51s/it]

...

 98%|█████████▊| 55/56 [1:23:08<01:30, 90.69s/it]

Train, epoch: [0][ 0/56]	Loss 4.7940e-01 (7.9119e-01)	Accuracy 0.864 (0.769)


100%|██████████| 56/56 [1:24:39<00:00, 90.71s/it]

Train, epoch: [0][ 0/56]	Loss 4.4236e-01 (7.8496e-01)	Accuracy 0.864 (0.771)
classes           IoU
---------------------
Pole          : 0.001
SignSymbol    : 0.000
Bicyclist     : 0.001
Pedestrian    : 0.003
Building      : 0.685
Fence         : 0.025
Pavement      : 0.465
Road          : 0.820
Car           : 0.518
Sky           : 0.773
Tree          : 0.558
---------------------
Mean IoU      : 0.350
---------------------


NameError: name 'datetime' is not defined

### test

In [25]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=1, shuffle=True)

In [26]:
# Load best model

# save path
if not os.path.isdir(os.getcwd() + '/result'):
    os.makedirs(os.getcwd() + '/result')

save_path = os.getcwd() + '/result/'
result = sorted(os.listdir(save_path), reverse=True)

In [27]:
result

['210601_21:02_checkpoint.pth.tar',
 '210601_21:02_best_weights.pth.tar',
 '210531_21:37_checkpoint.pth.tar',
 '210531_21:37_best_weights.pth.tar']

In [28]:
checkpoint = torch.load(save_path + result[1]) # 가장 최신 best_weights 파일 가져옴
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
print('Loaded best model weights (epoch {}) from {}'.format(checkpoint['epoch'], save_path + result[1]))

Loaded best model weights (epoch 0) from /home/ag/Capstone2/TEST/result/210601_21:02_best_weights.pth.tar


In [29]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
iou = iouCalc(classLabels, validClasses, voidClass = 11)
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

100%|██████████| 141/141 [11:05<00:00,  4.72s/it]

classes           IoU
---------------------
Pole          : 0.000
SignSymbol    : 0.043
Bicyclist     : 0.003
Pedestrian    : 0.131
Building      : 0.696
Fence         : 0.000
Pavement      : 0.127
Road          : 0.000
Car           : 0.055
Sky           : 0.872
Tree          : 0.710
---------------------
Mean IoU      : 0.240
---------------------
loss : 2.9336 acc : 0.4553 miou : 0.2398
